# 환율 기사 불용어 제거 및 전처리 및 기사 본문 요약

# 0. Library

In [1]:
import pandas as pd
import os, gc, re
from tqdm import tqdm
from datetime import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. 불용어 제거/전처리

In [1]:
INPUT_FILES = [
    "naver_finance_news_2020_labeled.csv",
    "naver_finance_news_2021_labeled.csv",
    "naver_finance_news_2022_labeled.csv",
    "naver_finance_news_2023_labeled.csv",  
    "naver_finance_news_2024_labeled.csv",
]

def clean_content(text: str) -> str:
    """뉴스 본문 불용어/잡음 제거"""
    if not isinstance(text, str):
        return ""

    # 1. 기자명 + =(서울=연합뉴스) ... 기자 =
    text = re.sub(r"\([^)]*연합뉴스[^)]*\)\s*[^=]*기자\s*=", " ", text)

    # 2. 이메일 (abc@yna.co.kr)
    text = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", " ", text)

    # 3. ▶, ▷, 【 】 같은 특수 기호로 시작하는 홍보/구독 문구
    text = re.sub(r"[▶▷【].*?(구독|제보|클릭|만나보세요|채널).*", " ", text)

    # 4. 사진/그래픽/자료/제공 표기
    text = re.sub(r"(사진=|그래픽=|자료사진|사진공동취재단|연합뉴스TV제공)", " ", text)

    # 5. 재판매 금지/DB 금지
    text = re.sub(r"(재판매\s*및\s*DB\s*금지)", " ", text)

    # 6. 대괄호 [] 안의 출처 (예: [연합뉴스 제공])
    text = re.sub(r"\[[^\]]*(연합뉴스|제공)[^\]]*\]", " ", text)

    # 7. 여러 개 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text

def process_file(fp: str):
    if not os.path.exists(fp):
        print(f"[WARN] 파일 없음: {fp}")
        return

    df = pd.read_csv(fp, encoding="utf-8-sig")
    if "content" not in df.columns:
        print(f"[WARN] {fp}: 'content' 컬럼 없음 → 스킵")
        return

    before_len = df["content"].str.len().sum()

    df["content"] = df["content"].apply(clean_content)

    after_len = df["content"].str.len().sum()
    print(f"[INFO] {fp}: 불용어 제거 완료 (총 글자수 {before_len} → {after_len})")

    out_fp = fp.replace(".csv", "_cleaned.csv")
    df.to_csv(out_fp, index=False, encoding="utf-8-sig")
    print(f"[INFO] 저장: {out_fp}")

for f in INPUT_FILES:
    process_file(f)

[INFO] naver_finance_news_2020_labeled.csv: 불용어 제거 완료 (총 글자수 4779800 → 4038476)
[INFO] 저장: naver_finance_news_2020_labeled_cleaned.csv
[INFO] naver_finance_news_2021_labeled.csv: 불용어 제거 완료 (총 글자수 5219578 → 4634627)
[INFO] 저장: naver_finance_news_2021_labeled_cleaned.csv
[INFO] naver_finance_news_2022_labeled.csv: 불용어 제거 완료 (총 글자수 6160397 → 5770330)
[INFO] 저장: naver_finance_news_2022_labeled_cleaned.csv
[INFO] naver_finance_news_2023_labeled.csv: 불용어 제거 완료 (총 글자수 7320934 → 6823222)
[INFO] 저장: naver_finance_news_2023_labeled_cleaned.csv
[INFO] naver_finance_news_2024_labeled.csv: 불용어 제거 완료 (총 글자수 4418631.0 → 4132446)
[INFO] 저장: naver_finance_news_2024_labeled_cleaned.csv


In [2]:
# --- 컴파일된 정규식 (속도/가독성) ---
RE_EMAIL = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
# 바로 뒤가 이메일이면 앞의 날짜 스탬프도 같이 삭제 (예: 2021.12.29saba@...)
RE_DATE_BEFORE_EMAIL = re.compile(r"\b(?:19|20)\d{2}\.\d{1,2}\.\d{1,2}(?=\s*[A-Za-z0-9._%+-]+@)")

# (서울=연합뉴스) / (부산=연합뉴스) ... 등 괄호 출처
RE_PAREN_YNA = re.compile(r"\([^)]*연합뉴스[^)]*\)")
# (=연합뉴스) 형태까지 커버
RE_PAREN_EQ_YNA = re.compile(r"\([^)]*=\s*연합뉴스[^)]*\)")

# 기자 표기(variations): "홍길동 기자 =", "홍길동 기자=", "기자 ="
RE_REPORTER_EQ = re.compile(r"[가-힣A-Za-z.\s]{1,30}기자\s*=")

# 대괄호 출처/제공(자료사진/연합뉴스/제공 등)
RE_BRACKET_SOURCE = re.compile(r"\[[^\]]*(연합뉴스|연합뉴스TV|제공|자료사진|사진공동취재단)[^\]]*\]")

# 캡션/크레딧 상투구
RE_CREDIT_PHRASES = re.compile(
    r"(사진\s*=\s*연합뉴스|그래픽\s*=\s*연합뉴스|자료사진|연합뉴스TV제공|사진공동취재단|재판매\s*및\s*DB\s*금지)"
)

# 홍보 꼬리말(▶, ▷, 【 … 구독/제보/클릭/만나보세요/채널 …)
RE_PROMO_TAIL = re.compile(r"[▶▷【].*?(구독|제보|클릭|만나보세요|채널).*")

# 라벨/괄호 뒤 군더더기 콤마 정리용
RE_COMMA_GAP = re.compile(r"\s*,\s*")

def clean_content(text: str) -> str:
    if not isinstance(text, str):
        return ""
    s = text

    # 1) 날짜스탬프(YYYY.MM.DD) + 붙은 이메일 같이 제거
    s = RE_DATE_BEFORE_EMAIL.sub(" ", s)

    # 2) 이메일 전부 제거
    s = RE_EMAIL.sub(" ", s)

    # 3) 괄호 출처(연합뉴스)류 제거
    s = RE_PAREN_YNA.sub(" ", s)
    s = RE_PAREN_EQ_YNA.sub(" ", s)

    # 4) 기자= 패턴 제거
    s = RE_REPORTER_EQ.sub(" ", s)

    # 5) 대괄호 출처/제공 제거
    s = RE_BRACKET_SOURCE.sub(" ", s)

    # 6) 캡션/크레딧 상투구 제거
    s = RE_CREDIT_PHRASES.sub(" ", s)

    # 7) 홍보 꼬리말 제거
    s = RE_PROMO_TAIL.sub(" ", s)

    # 8) 여분의 공백/콤마 정리
    s = RE_COMMA_GAP.sub(", ", s)
    s = re.sub(r"\s+", " ", s).strip()
    # 문장 앞뒤 구두점 정리
    s = re.sub(r"^\s*[,.;:]\s*", "", s)
    s = re.sub(r"\s*[,.;:]\s*$", "", s)

    return s

def process_file(fp: str):
    if not os.path.exists(fp):
        print(f"[WARN] 파일 없음: {fp}")
        return
    df = pd.read_csv(fp, encoding="utf-8-sig", engine="python")
    df = df.rename(columns={c: c.strip() for c in df.columns})

    if "content" not in df.columns:
        print(f"[WARN] {fp}: 'content' 컬럼 없음 → 스킵")
        return

    before = df["content"].astype(str).str.len().sum()
    df["content"] = df["content"].apply(clean_content)
    after = df["content"].astype(str).str.len().sum()

    out_fp = fp.replace(".csv", "_cleaned.csv")
    df.to_csv(out_fp, index=False, encoding="utf-8-sig")
    print(f"[INFO] {fp} → {out_fp} | 글자수 {before} → {after}")

# 실행
for f in INPUT_FILES:
    process_file(f)

[INFO] naver_finance_news_2020_labeled.csv → naver_finance_news_2020_labeled_cleaned.csv | 글자수 4779800 → 4017921
[INFO] naver_finance_news_2021_labeled.csv → naver_finance_news_2021_labeled_cleaned.csv | 글자수 5219578 → 4628637
[INFO] naver_finance_news_2022_labeled.csv → naver_finance_news_2022_labeled_cleaned.csv | 글자수 6160397 → 5732214
[INFO] naver_finance_news_2023_labeled.csv → naver_finance_news_2023_labeled_cleaned.csv | 글자수 7320934 → 6775183
[INFO] naver_finance_news_2024_labeled.csv → naver_finance_news_2024_labeled_cleaned.csv | 글자수 4418634 → 4105885


# 2. 환율 기사 요약

In [1]:
# ===== cleaned 파일만 사용 =====
YEAR_FILES = {
    2020: "naver_finance_news_2020_labeled_cleaned.csv",
    2021: "naver_finance_news_2021_labeled_cleaned.csv",
    2022: "naver_finance_news_2022_labeled_cleaned.csv",
    2023: "naver_finance_news_2023_labeled_cleaned.csv",
    2024: "naver_finance_news_2024_labeled_cleaned.csv",
}

# 출력 경로
MERGED_CSV = "naver_finance_news_2020_2024_merged_cleaned.csv"
OUTPUT_CSV = "naver_finance_news_2020_2024_with_kobart.csv"
FINAL_CSV  = "naver_finance_news_2020_2024_with_kobart_v2.csv"

# ===== 요약 파라미터 =====
MODEL_NAME      = "EbanLee/kobart-summary-v3"
TEXT_COL        = "content"
OUT_COL         = "summary_kobart_v3"

ENC_MAX_TOKENS  = 1024
GEN_MAX_TOKENS  = 96
NUM_BEAMS       = 5
LENGTH_PENALTY  = 1.2
NO_REPEAT_NGRAM = 3

# reduce 단계 (map-reduce 최종 축약)
REDUCE_ENC_MAX  = 768
REDUCE_GEN_MAX  = 120
REDUCE_BEAMS    = 5

BATCH_SIZE_SHORT = 8
SHORT_THRESH     = ENC_MAX_TOKENS  # 1024 토큰 이하면 "short"

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
def parse_date_any(x):
    s = str(x).strip()
    for fmt in ("%Y%m%d", "%Y-%m-%d", "%Y/%m/%d", "%Y.%m.%d"):
        try:
            return datetime.strptime(s, fmt).date().isoformat()
        except Exception:
            pass
    try:
        return pd.to_datetime(s).date().isoformat()
    except Exception:
        return s

dfs = []
for y in sorted(YEAR_FILES):
    fp = YEAR_FILES[y]
    if not os.path.exists(fp):
        print(f"[WARN] 파일 없음: {fp}")
        continue
    df = pd.read_csv(fp, encoding="utf-8-sig", engine="python")
    df = df.rename(columns={c: c.strip().lower() for c in df.columns})
    for c in ["date","title","url","content"]:
        if c not in df.columns:
            df[c] = ""
    df["date"] = df["date"].apply(parse_date_any)
    df["year"] = y
    dfs.append(df)

if not dfs:
    raise RuntimeError("병합할 cleaned 파일을 찾지 못했습니다.")

merged = pd.concat(dfs, ignore_index=True)

# 날짜 정렬 및 중복 제거
merged["__sort_date"] = pd.to_datetime(merged["date"], errors="coerce")
merged = merged.sort_values(by=["year","__sort_date"]).drop(columns=["__sort_date"])
merged = merged.drop_duplicates(subset=["date","title","url","content"], keep="first").reset_index(drop=True)

merged.to_csv(MERGED_CSV, index=False, encoding="utf-8-sig")
print(f"[INFO] 병합 완료 → {MERGED_CSV}, 행수={len(merged)}")
merged.head(2)

[INFO] 병합 완료 → naver_finance_news_2020_2024_merged_cleaned.csv, 행수=35634


,date,title,url,content,category,confidence,year
0,2020-01-01,"과천시, 새해 맞아 내달 지역화폐 10% 할인 판매",https://n.news.naver.com/mnews/article/001/001...,경기 과천시는 2020년 새해를 맞아 1월 한 달 간 지역화폐인 '과천토리'를 10...,Irrelevant,0.95,2020
1,2020-01-01,브라질 헤알화 가치 신흥국 통화 중 선방…경제회복 효과,https://n.news.naver.com/mnews/article/001/001...,아르헨·터키·칠레 등과 비교해 하락폭 적어 김재순 특파원 = 브라질 헤알화의 가치가...,Irrelevant,0.85,2020


In [3]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device).eval()

def tokenize_len(text: str) -> int:
    return len(tok.encode(str(text), add_special_tokens=True, truncation=False))

def chunk_by_tokens(text: str, max_tokens: int):
    ids = tok.encode(str(text), add_special_tokens=False, truncation=False)
    if not ids:
        return [""]
    return [tok.decode(ids[i:i+max_tokens], skip_special_tokens=True)
            for i in range(0, len(ids), max_tokens)]

@torch.no_grad()
def generate_batch(batch_texts, enc_max, gen_max, beams, lp, ngram, device):
    if not batch_texts:
        return []
    enc = tok(batch_texts, return_tensors="pt", padding=True, truncation=True,
              max_length=enc_max).to(device)
    out = model.generate(
        **enc,
        max_length=gen_max,
        min_length=min(12, gen_max//2),
        num_beams=beams,
        length_penalty=lp,
        no_repeat_ngram_size=ngram,
    )
    texts = [tok.decode(o, skip_special_tokens=True).replace("\n"," ").strip() for o in out]
    del enc, out
    if device == "cuda":
        torch.cuda.empty_cache()
    gc.collect()
    return texts

def summarize_long(text: str) -> str:
    chunks = chunk_by_tokens(text, ENC_MAX_TOKENS)
    part_sums = generate_batch(
        chunks, ENC_MAX_TOKENS, GEN_MAX_TOKENS, NUM_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device
    )
    stitched = " ".join(part_sums)
    final = generate_batch(
        [stitched], REDUCE_ENC_MAX, REDUCE_GEN_MAX, REDUCE_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device
    )[0]
    return final

def summarize_short_batch(batch_texts):
    return generate_batch(
        batch_texts, ENC_MAX_TOKENS, REDUCE_GEN_MAX, NUM_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device
    )

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [5]:
df = pd.read_csv(MERGED_CSV, encoding="utf-8-sig", engine="python")

if TEXT_COL not in df.columns:
    raise RuntimeError(f"'{TEXT_COL}' 컬럼이 없습니다.")

texts = df[TEXT_COL].fillna("").astype(str).str.replace("\n", " ").str.strip()
mask  = texts.str.len() > 0

# 이어 달리기 지원: OUT_COL이 있으면 재사용
if OUT_COL in df.columns:
    summaries = df[OUT_COL].fillna("").astype(str).tolist()
else:
    summaries = [""] * len(df)

# 요약 대상만 토큰 길이 계산
lengths = [tokenize_len(t) if (mask.iloc[i] and not summaries[i]) else 0 for i, t in enumerate(texts)]

short_idxs = [i for i, l in enumerate(lengths) if l > 0 and l <= SHORT_THRESH]
long_idxs  = [i for i, l in enumerate(lengths) if l > SHORT_THRESH]

print(f"[INFO] 요약 대상: short {len(short_idxs)}건, long {len(long_idxs)}건")

# ---- 짧은 본문: 배치 처리
for i in tqdm(range(0, len(short_idxs), BATCH_SIZE_SHORT), desc="Summarizing (short)"):
    idx_batch = short_idxs[i:i+BATCH_SIZE_SHORT]
    batch_texts = [texts.iloc[k] for k in idx_batch]
    outs = summarize_short_batch(batch_texts)
    for j, k in enumerate(idx_batch):
        summaries[k] = outs[j] if j < len(outs) else ""

# ---- 긴 본문: map-reduce 처리
for k in tqdm(long_idxs, desc="Summarizing (long, map-reduce)"):
    if summaries[k]:  # 이미 요약된 경우 스킵(이어달리기)
        continue
    try:
        summaries[k] = summarize_long(texts.iloc[k])
    except torch.cuda.OutOfMemoryError:
        # GPU 메모리 부족 시 보수적으로 재시도
        torch.cuda.empty_cache()
        try:
            summaries[k] = summarize_long(texts.iloc[k])
        except Exception:
            summaries[k] = ""
    except Exception:
        summaries[k] = ""

df[OUT_COL] = summaries
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"[INFO] 1차 저장 완료: {OUTPUT_CSV}")

[INFO] 요약 대상: short 34704건, long 929건


Summarizing (long, map-reduce): 100%|████████████████████████████████████████████████| 929/929 [31:10<00:00,  2.01s/it]


[INFO] 1차 저장 완료: naver_finance_news_2020_2024_with_kobart.csv


In [6]:
df = pd.read_csv(OUTPUT_CSV, encoding="utf-8-sig", engine="python")

# 기존에 'summary'라는 별도 컬럼이 있으면 충돌 방지 차원에서 제거
if "summary" in df.columns and OUT_COL != "summary":
    df = df.drop(columns=["summary"])

df.to_csv(FINAL_CSV, index=False, encoding="utf-8-sig")
print(f"[완료] 최종 저장: {FINAL_CSV}, 행수={len(df)}")

df[["date","title",OUT_COL]].head(5)

[완료] 최종 저장: naver_finance_news_2020_2024_with_kobart_v2.csv, 행수=35634


,date,title,summary_kobart_v3
0,2020-01-01,"과천시, 새해 맞아 내달 지역화폐 10% 할인 판매",과천 지역화폐 새해 특별할인 과천토리는 평상시 1인당 월 40만원의 한도 내에서 6...
1,2020-01-01,브라질 헤알화 가치 신흥국 통화 중 선방…경제회복 효과,31일(현지시간) 브라질 일간 에스타두 지 상파울루에 따르면 지난 27일을 기준으로...
2,2020-01-01,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치(종합)",달러 외환당국이 지난 3분기(7~9월) 시장안정을 위해 외환시장에서 28억7천만달러...
3,2020-01-01,"[1보] 외환당국, 3분기 중 28억7천만달러 순매도","한국은행·기재부, 외환시장 개입내역 공개달러"
4,2020-01-01,2019년 달러 대비 엔환율 변동폭 6.82엔…1998년 이후 최소,도쿄 외환 시장에서 미 달러화 대비 엔화 환율의 등락폭이 올해 20여년 만에 가장 ...
